In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error

In [ ]:
import pickle
with open('/content/drive/MyDrive/final_learningset_SWdelay.pickle', 'rb') as f:
    [learnvector_o,learnvector_m,learnvector_s,learnvector,timevector]=pickle.load(f)

In [ ]:
import numpy as np
np.array(learnvector_o).T.shape

In [ ]:
input_x=np.array(learnvector)
target_y=np.array(timevector)[:,2]

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(15,7))
plt.plot(target_y)

In [ ]:
plt.hist(target_y/60)
plt.xlabel('Time delay(s)')
plt.ylabel('Frequency')

In [ ]:
##input data
plt.hist(np.array(learnvector_o).T[:,0],label='rx')

In [ ]:
plt.hist(np.array(learnvector_o).T[:,1],label='ry')

In [ ]:
plt.hist(np.array(learnvector_o).T[:,2],label='rz')

In [ ]:
plt.hist(np.array(learnvector_o).T[:,3],label='vx')

In [ ]:
plt.hist(np.array(learnvector_o).T[:,4],label='vy')

In [ ]:
plt.hist(np.array(learnvector_o).T[:,5],label='vz')

In [ ]:
##ML model
#standarsization done already
#Models-GPR,RF,Adaptive Boosting, Neural Network, Gaussian Naive Bayes, Quadratic Discriminant Analysis, and eXtreme Gradient Boosting, 

In [ ]:
#Random-splitting
import numpy as np
from sklearn.model_selection import train_test_split
#x_test,y_test=input_x[-80:],target_y[-80:]
x_train,x_test,y_train,y_test=train_test_split(input_x, target_y, test_size=0.20, random_state=42)

In [ ]:
print((x_train).shape,(x_test).shape,(y_train).shape,(y_test).shape)

In [ ]:
target_y=pd.DataFrame(target_y)

In [ ]:
x_1bin=[]
x_2bin=[]
x_3bin=[]
y_1bin=[]
y_2bin=[]
y_3bin=[]

for i in range(0,len(target_y)):
  if target_y.iloc[i,0]<2000:
    x_1bin.append(input_x[i])
    y_1bin.append(target_y.iloc[i,0])
  elif (target_y.iloc[i,0]>2000 and target_y.iloc[i,0]<3500):
      x_2bin.append(input_x[i])
      y_2bin.append(target_y.iloc[i,0])
  else:
      x_3bin.append(input_x[i])
      y_3bin.append(target_y.iloc[i,0])


In [ ]:
def pred_bins(model):
  pred_1bin=model.predict(np.array(x_1bin))
  pred_2bin=model.predict(np.array(x_2bin))
  pred_3bin=model.predict(np.array(x_3bin))
  #error
  bin1_error=(np.sqrt(mean_squared_error(np.array(y_1bin),pred_1bin))/60)
  bin2_error=(np.sqrt(mean_squared_error(np.array(y_2bin),pred_2bin))/60)
  bin3_error=(np.sqrt(mean_squared_error(np.array(y_3bin),pred_3bin))/60)
  return pred_1bin,pred_2bin,pred_3bin,bin1_error,bin2_error,bin3_error
  


In [ ]:
plt.hist(y_train)

In [ ]:
plt.hist(y_test)

In [ ]:
##feature selection
from xgboost import XGBClassifier
from xgboost import plot_importance
from matplotlib import pyplot

fi = XGBClassifier()
fi.fit(x_train,y_train)
# plot feature importance
plot_importance(fi)
pyplot.show()

In [ ]:
def get_models():
	models = list()
	models.append(LogisticRegression())
	models.append(Ridge(alpha=1.0))
	models.append(GaussianProcessRegressor())
  models.append(RandomForestClassifier(n_estimators=150))
  models.append(XGBRegressor())
  models.append(SVR(kernel = 'rbf'))
  models.append(DecisionTreeRegressor(random_state = 0))
  models.append

	
	return models

In [ ]:
#inputs
input_x
target_y

In [ ]:
# prepare the cross-validation procedure
def cv(model,x_1bin,x_2bin,x_3bin,y_1bin,y_2bin,y_3bin):
  cv = KFold(n_splits=3, random_state=1, shuffle=True)
  # create model

  # evaluate model
  scores1 = cross_val_score(model, x_1bin, np.array(y_1bin), scoring='neg_mean_squared_error', cv=cv, n_jobs=-1)
  scores2 = cross_val_score(model, x_2bin, np.array(y_2bin), scoring='neg_mean_squared_error', cv=cv, n_jobs=-1)
  scores3 = cross_val_score(model, x_3bin, np.array(y_3bin), scoring='neg_mean_squared_error', cv=cv, n_jobs=-1)
  print('Bin1 : MSE: %.3f (%.3f)' % ((np.sqrt(np.abs(mean(scores1))))/60, (np.sqrt(std(scores1)))/60))
  print('Bin2 : MSE: %.3f (%.3f)' % ((np.sqrt(np.abs(mean(scores2))))/60, (np.sqrt(std(scores2)))/60))
  print('Bin3 : MSE: %.3f (%.3f)' % ((np.sqrt(np.abs(mean(scores3))))/60, (np.sqrt(std(scores3)))/60))

In [ ]:
##define models
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Ridge
from numpy import mean
from numpy import std
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor 
from sklearn.ensemble import AdaBoostRegressor
rdge = Ridge(alpha=1.0)
gpr=GaussianProcessRegressor()
rfc=RandomForestClassifier(n_estimators=150)
DT = DecisionTreeRegressor(random_state = 0)
adaboost_model= AdaBoostRegressor()
svre = SVR(kernel = 'rbf')
xgbr = XGBRegressor(n_estimators=200, max_depth=3, eta=0.1, subsample=0.7, colsample_bytree=0.8)


In [ ]:
cv(xgbr,x_1bin,x_2bin,x_3bin,y_1bin,y_2bin,y_3bin)

In [ ]:
mean_b1=[3.779,19.064,4.641,5.297,3.698,4.073,3.680]
mean_b2=[4.360,11.014,7.194,5.878,4.486,6.946,4.764   ]
mean_b3=[4.137,14.514 ,7.202,5.325,4.228,6.146,4.222   ]
std_b1=[1.160,6.891,1.349,2.920,1.152,1.762,1.129]
std_b2=[1.366,6.092,3.025,1.531,1.685,1.658,2.040]
std_b3=[1.825,8.266,3.661,2.680,1.865,3.852,1.497]
model_n=['Ridge','GPR','RF','DT','Adaboost','SVR','XGB']

In [ ]:
mean_b1=[3.779,4.36,4.137]


In [ ]:
yerr.T

In [ ]:
df = pd.DataFrame([[3.779,4.36,4.137],[19.06,11.01,14.51],[4.64,7.19,7.20],[5.29,5.87,5.32],[3.69,4.4,4.22],[4.07,6.9,6.14],[3.68,4.76,4.22]], columns = ['mean_VHS', 'mean_HS', 'mean_SS'], index=['Ridge','GPR','RF','DT','Adaboost','SVR','XGB'])
df2=pd.DataFrame([[1.16,1.36,1.82],[6.89,6.09,8.26],[1.3,3.0,3.66],[2.92,1.53,2.68],[1.15,1.68,1.86],[1.76,1.65,3.85],[1.12,2.04,1.49]],columns=['stdb1','stdb2','stdb3'],index=['Ridge','GPR','RF','DT','Adaboost','SVR','XGB'])
# convert the std columns to an array
#std_b1_b2_b3=pd.DataFrame([[std_b1.T,std_b2.T,std_b3.T]],columns=['stdb1','stdb2','stdb3'])
#yerr = std_b1_b2_b3[['stdb1','stdb2','stdb3']].to_numpy().T

# print(yerr)
yerr = df2[['stdb1', 'stdb2','stdb3']].to_numpy().T

plt.figure(figsize=(10,7))
df[['mean_VHS', 'mean_HS', 'mean_SS']].plot(kind='bar', yerr=yerr, alpha=0.5, error_kw=dict(ecolor='k'),figsize=(10, 7))
#df.plot(figsize=(20, 10))
#plt.xlabel('xlabel')
plt.ylabel('delay error (minutes)')
plt.show()

In [ ]:
plt.errorbar(model_n, mean_b1, yerr=std_b1, label='both limits (default)', fmt='.k')
plt.errorbar(model_n, mean_b2, yerr=std_b2, label='both limits (default)')
plt.errorbar(model_n, mean_b3, yerr=std_b3, label='both limits (default)')

In [ ]:
#y=np.range(0,20,1)
fig = plt.figure(figsize=(11.5,6))
plt.ylim([0,20])
plt.bar(model_n,mean_b1,width=0.4,color='blue',alpha=0.3,edgecolor='black')
plt.bar(model_n,mean_b2,width=0.4,color='orange',alpha=0.3,edgecolor='black')
plt.bar(model_n,mean_b3,width=0.4,color='yellow',alpha=0.3,edgecolor='black')
fig.suptitle('Accuracy Scores', fontsize=20)
plt.xlabel('ML_model',fontsize=16)
plt.ylabel('delay_error(mins)',fontsize=16)


In [ ]:
cv = KFold(n_splits=3, random_state=1, shuffle=True)
  # create model

  # evaluate model
scores1 = cross_val_score(rdge, np.array(x_1bin), np.array(y_1bin), scoring='neg_mean_squared_error', cv=cv, n_jobs=-1)
print('Bin1 : MSE: %.3f (%.3f)' % (np.abs(mean(scores1)), std(scores1)))

In [ ]:
rf_1bin,rf_2bin,rf_3bin,rf1_error,rf2_error,rf3_error=pred_bins(clf1)
print(rf1_error,rf2_error,rf3_error)


In [ ]:
x_train.shape

In [ ]:
np.array(x_3bin).shape

In [ ]:
##ridge regression
from sklearn.linear_model import Ridge
rdge = Ridge(alpha=1.0)
rdge.fit(x_train,y_train)


In [ ]:
##GPR(Gaussian process regressor)

from sklearn.gaussian_process import GaussianProcessRegressor

gpr=GaussianProcessRegressor()
gpr.fit(x_train,y_train)
y_gpr=gpr.predict(x_test)
#error
y_gpr_error=np.sqrt(mean_squared_error(y_test,y_gpr))
print(y_gpr_error/60,'mins')

In [ ]:
#Randomforest
from sklearn.ensemble import RandomForestClassifier
clf1=RandomForestClassifier(n_estimators=150)
clf1.fit(x_train,y_train)

In [ ]:
y_pred1=clf1.predict(x_test)

In [ ]:
plt.figure(figsize=(15,7))
#plt.plot(y_pred,label='100trees')
plt.plot(y_test,label='observed')
plt.plot(y_pred1,label='50trees')
plt.legend()

In [ ]:
#error
y_rf_error=np.sqrt(mean_squared_error(y_test,y_pred1))
print(y_rf_error/60,'mins')

In [ ]:
plt.figure(figsize=(7,7))
plt.scatter(y_test,y_pred)
plt.xlabel('observed')
plt.ylabel('predicted')

In [ ]:
##predicting with bins

rf_1bin,rf_2bin,rf_3bin,rf1_error,rf2_error,rf3_error=pred_bins(clf1)
print(rf1_error,rf2_error,rf3_error)


In [ ]:
#2)XGB
#eXtreme Gradient Boosting, 
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
model = XGBRegressor()
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
model.fit(x_train, y_train)
# force scores to be positive


In [ ]:
y_xgboost = model.predict(x_test)

In [ ]:
plt.figure(figsize=(7,7))
plt.scatter(y_test,y_xgboost)
plt.xlabel('observed')
plt.ylabel('predicted')

In [ ]:
#error
y_xg_error=np.sqrt(mean_squared_error(y_test,y_xgboost))
print('y_xg_error:',y_xg_error/60,'mins')

In [ ]:
xg_1bin,xg_2bin,xg_3bin,xg1_error,xg2_error,xg3_error=pred_bins(model)
print(xg1_error,xg2_error,xg3_error)

In [ ]:
##SVR
from sklearn.svm import SVR
regressor = SVR(kernel = 'rbf')
regressor.fit(x_train, y_train)
y_svr = regressor.predict(x_test)

In [ ]:
plt.figure(figsize=(7,7))
plt.scatter(y_test,y_svr)
plt.xlabel('observed')
plt.ylabel('predicted')

In [ ]:
#error
y_svr_error=np.sqrt(mean_squared_error(y_test,y_svr))
print('y_svr_error:',y_svr_error/60,'mins')

In [ ]:
svr_1bin,svr_2bin,svr_3bin,svr1_error,svr2_error,svr3_error=pred_bins(regressor)
print(svr1_error,svr2_error,svr3_error)

In [ ]:
##decisiontree
from sklearn.tree import DecisionTreeRegressor 
DT = DecisionTreeRegressor(random_state = 0)
DT.fit(x_train, y_train)
y_dt=DT.predict(x_test)

In [ ]:
plt.figure(figsize=(7,7))
plt.scatter(y_test,y_dt)
plt.xlabel('observed')
plt.ylabel('predicted')

In [ ]:
#error
y_dt_error=np.sqrt(mean_squared_error(y_test,y_dt))
print('y_dt_error:',y_dt_error/60,'mins')

In [ ]:
DT_1bin,DT_2bin,DT_3bin,DT1_error,DT2_error,DT3_error=pred_bins(DT)
print(DT1_error,DT2_error,DT3_error)

In [ ]:
##adaboost regression
from sklearn.ensemble import AdaBoostRegressor
adaboost_model= AdaBoostRegressor()
adaboost_model.fit(x_train, y_train)
y_adaboost=adaboost_model.predict(x_test)

In [ ]:
#error
y_adaboost_error=np.sqrt(mean_squared_error(y_test,y_adaboost))
print('y_adaboost_error:',y_adaboost_error/60,'mins')

In [ ]:
plt.figure(figsize=(7,7))
plt.scatter(y_test,y_adaboost)
plt.xlabel('observed')
plt.ylabel('predicted')

In [ ]:
AB_1bin,AB_2bin,AB_3bin,AB1_error,AB2_error,AB3_error=pred_bins(adaboost_model)
print(AB1_error,AB2_error,AB3_error)

In [ ]:
#neural networks
from keras.layers import Dense, BatchNormalization, Dropout, LSTM
from keras.models import Sequential

from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from tensorflow.keras import metrics
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
from keras import callbacks


def ann_model():
  model_nn=Sequential()
  model_nn.add(Dense(units = 32, activation = 'relu', input_dim = 6))
  model_nn.add(Dense(units = 16, activation = 'relu'))
  #model_nn.add(Dropout(0.25))
  #model_nn.add(Dense(units = 8, kernel_initializer = 'uniform', activation = 'relu'))
  model_nn.add(Dense(units = 1, activation = 'linear'))


  model_nn.compile(optimizer = Adam(learning_rate=0.001), loss = 'mse', metrics = [metrics.mean_squared_error])
  return model_nn

In [ ]:
history=model_nn.fit(x_train,y_train,batch_size = 16, epochs = 200, validation_split=0.2)

In [ ]:
import pandas as pd
history_df = pd.DataFrame(history.history)

plt.plot(history_df.loc[:, ['loss']], label='Training loss')
plt.plot(history_df.loc[:, ['val_loss']], label='Validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc="best")

In [ ]:
y_nn=model_nn.predict(x_test)

In [ ]:
#error
y_nn_error=np.sqrt(mean_squared_error(y_test,y_nn))
print('y_nn_error:',y_nn_error/60,'mins')

In [ ]:
NN_1bin,NN_2bin,NN_3bin,NN1_error,NN2_error,NN3_error=pred_bins(model_nn)
print(NN1_error,NN2_error,NN3_error)

In [ ]:
plt.figure(figsize=(7,7))
plt.scatter(y_test,y_nn)
plt.xlabel('observed')
plt.ylabel('predicted')

In [ ]:
plt.figure(figsize=(15,7))
plt.plot(target_y)
plt.plot(model_nn.predict(input_x))